In [1]:
import pandas as pd
import geopandas as gpd
from census import Census
from us import states

c = Census("771ac44f4c55670ec890af7951073be949e5130c")

https://pygis.io/docs/d_access_census.html

https://api.census.gov/data/2020/acs/acs5/variables.html

https://api.census.gov/data/2020/acs/acs5/subject/variables.html

https://unicede.air-worldwide.com/unicede/unicede_washington_fips.html

In [22]:
COUNTIES = ("001", "003", "019", "023", "043", "051", "063", "065", "075")


In [23]:
acs5_data = c.acs5.state_county_tract(
    fields=(
        "NAME",
        "B28003_002E",
        "B28003_006E",
        "B28011_002E",
        "B28011_008E",
        "B01003_001E",
        "B08101_049E",
    ),
    state_fips=states.WA.fips,
    county_fips="*",
    tract="*",
    year=2020,
)


In [24]:
acs5s_data = c.acs5st.state_county_tract(
    fields=(
        "NAME",
        "S0101_C01_030E",
        "S1901_C01_012E",
    ),
    state_fips=states.WA.fips,
    county_fips="*",
    tract="*",
    year=2020,
)


In [25]:
variable_data = acs5_data + acs5s_data


In [26]:
acs5_data = pd.DataFrame(acs5_data)
acs5s_data = pd.DataFrame(acs5s_data)


In [27]:
data = pd.merge(acs5_data, acs5s_data)

data["GEOID"] = data.state + data.county + data.tract

data = data[
    [
        "NAME",
        "GEOID",
        "tract",
        "B28003_002E",
        "B28003_006E",
        "B28011_002E",
        "B28011_008E",
        "B01003_001E",
        "B08101_049E",
        "S0101_C01_030E",
        "S1901_C01_012E",
        "state",
        "county",
    ]
]

data = data[data.county.isin(COUNTIES)].drop(columns=["state", "county"])

data.head()


,NAME,GEOID,tract,B28003_002E,B28003_006E,B28011_002E,B28011_008E,B01003_001E,B08101_049E,S0101_C01_030E,S1901_C01_012E
0,"Census Tract 9601, Asotin County, Washington",53003960100,960100,1664.0,118.0,1569.0,183.0,4320.0,145.0,1337.0,73942.0
1,"Census Tract 9602, Asotin County, Washington",53003960200,960200,1872.0,142.0,1780.0,222.0,4755.0,421.0,1352.0,72988.0
2,"Census Tract 9603, Asotin County, Washington",53003960300,960300,1279.0,85.0,1156.0,192.0,3633.0,6.0,396.0,38077.0
3,"Census Tract 9604, Asotin County, Washington",53003960400,960400,941.0,53.0,805.0,180.0,2517.0,29.0,647.0,38777.0
4,"Census Tract 9605, Asotin County, Washington",53003960500,960500,1345.0,181.0,1198.0,304.0,3517.0,26.0,635.0,42639.0


In [28]:
data.columns.tolist()


['NAME',
 'GEOID',
 'tract',
 'B28003_002E',
 'B28003_006E',
 'B28011_002E',
 'B28011_008E',
 'B01003_001E',
 'B08101_049E',
 'S0101_C01_030E',
 'S1901_C01_012E']

In [29]:
data = data.rename(
    columns={
        "B28003_002E": "has_computer",
        "B28003_006E": "no_computer",
        "B28011_002E": "with_internet",
        "B28011_008E": "no_internet",
        "B01003_001E": "total_population",
        "B08101_049E": "work_from_home",
        "S0101_C01_030E": "65_and_older",
        "S1901_C01_012E": "median_income",
    }
)

In [35]:
data.isna().any()

NAME                False
GEOID               False
tract               False
has_computer        False
no_computer         False
with_internet       False
no_internet         False
total_population    False
work_from_home      False
65_and_older        False
median_income       False
dtype: bool

In [31]:
data.to_csv("../data/census_data.csv", index=False)